In [1]:
Q = QQ['x']
x = Q.gen()

############################################# INPUT #########################################################

#SETTING OF THE MINIMAL POLYNOMIAL

# f = x**3-x**2-x-1 #d-Bonnaci number
# f = x**2-x-1   #Golden ratio
# f = x**2-3*x+1 #second power of the golden ratio
#f = x**2+4      #2i
# f = x**2+3*x+3 #Eisenstein
f = x**2-2*x-1 #Silver ratio
# f = x**2 +2*x+2 # Penney

#SETTING OF THE ALPHABET
A = [] #alphabet
# A = [0, 1, -1]
A = [0, 1, 2, -1, -2]
# A = [0,1,2,3,-1,-2,-3]
#A = [0,1,2,3,4,5,6,7,-1,-2,-3,-4,-5,-6,-7]
# A = [0,1,2,3,4,5,-1,-2,-3,-4,-5]
# A = [0,1,2,3,4,5,6,7,8,9,1,11,12,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12]


print " v = omega, w = omega^2"
print "Alphabet A = ", A
print ""

###############################################################################################################

print "Is irreducible: ", f.is_irreducible() #returns if the polynomial f is ireducible or not

N.<beta> = NumberField(f) #returns a root closes to the embedding
print N #returns what is N and what is beta

roots = beta.complex_embeddings()
print roots #returns roots of the polynomial f

numOutK = 0
position = 0
for i in range(f.degree()):
    if abs(roots[i])>=1:
        numOutK = numOutK + 1
        position = i
#         print position
pisot = 0
cPisot = 0

if (numOutK == 2) & (not roots[position].is_real()): #finds out if beta is Pisot or complex Pisot or neither
    print("Beta is complex Pisot")
    cPisot = 1
    position = position - 1
elif (numOutK == 1) & roots[position].is_real() & (beta > 1):
    print("Beta is Pisot")
    pisot = 1
else: 
    print("Beta is not Pisot or complex Pisot")
    
print N.places()
# position = 0                       #only if N acts unexpectedly
beta_complex = N.places()[position]  
print "beta = ", beta_complex(beta)

# A = [0, 1, -1, beta+1, -beta-1] #alphabet for Penney
# A = [0, 1, -1, beta+1, -beta-1, 1/(beta+1), -1/(beta+1)] #alphabet for Eisenstein

print "Alphabet A = ", A

def createString(k, i): #creates a list from D[k] and digit i
    aux = list(SS[k])
    aux = list([i] + aux[0:numPositions-2])
    return list(aux) 

def createRule(firstList, secondList): #function which concatenate two lists in rewriting rule
    fList = ""
    sList = ""
    for i in range(1,len(firstList)+1):
        if firstList[len(firstList) - i] == beta + 1:
            fList = fList + "v"
        elif firstList[len(firstList) - i] == -beta -1:
            fList = fList + "-v"
        elif firstList[len(firstList) - i] == 1/(beta+1):
            fList = fList + "w"
        elif firstList[len(firstList) - i] == -1/(beta+1):
            fList = fList + "-w"
        else:
            fList = fList + str(firstList[len(firstList) - i])
    for j in range(1,len(secondList)+1):
        if secondList[len(secondList) -j] == beta + 1:
            sList = sList + "v"
        elif secondList[len(secondList) -j] == -beta -1:
            sList = sList + "-v"
        elif secondList[len(secondList) -j] == 1/(beta+1):
            sList = sList + "w"
        elif secondList[len(secondList) -j] == -1/(beta+1):
            sList = sList + "-w"
        else:
            sList = sList + str(secondList[len(secondList) -j])
    return "".join(fList) + " -> " + "".join(sList)

def writeRule(j, k, i, degree): #writes rewriting rule into appropriate sets 
    aux = list(SS[k])
    LL.append(createRule(list([i] + aux[0:degree]),SS[spectrum.index(j)][:degree+1]))
    L.append([beta*spectrum[k]+i, degree+1])
    LonlyPositive.append([beta*spectrum[k]+i, degree+1])

def addNewElement(k, i, degree): #adds a new element to the spectrum and checks if it can be rewritten or not
    string = createString(k, i)
    if controlNewElement(degree, string) == True:
        setNonRewritable.append(beta_complex((beta*spectrum[k]+i)/(beta**(degree+1)))) #adds to the set value 0.abc ...
        stringNonRewritable.append(string)
    spectrum.append(beta*spectrum[k]+i) #add of new element into the set spectrum
    SS.append(string)

def deleteRewritableElement(index): #delete of the element on the position index from the set setNonRewritable
    setNonRewritable.pop(index)
    stringNonRewritable.pop(index)
       
def controlNewElement(degree, string): #returns TRUE for elements which can not be rewritten and FALSE otherwise
    if len(L)!=0:
        extrems = [[min(column), max(column)] for column in zip(*L)] 
        shortestRule = extrems[1][0]
        longestRule = extrems[1][1]
        for o in range(shortestRule, longestRule +1): #iteration over the size of the rewriting rules
            value = 0
            for p in range(o):
                if degree - p >= 0:
                    value = value * beta + string[degree-p]
                else:
                    value = value * beta
            if ([value, o] in L) or ([-value,o] in L):
                return False
    return True

def controlAllElements():#checks if elements in set setNonRewritable can not be rewritten while adding 0 to the end
    aux = [i for i in stringNonRewritable]
    for z in aux:
        for y in range(numPositions-1):
            if z[numPositions-2-y]!=0:
                degreeElement = numPositions -2- y
                break
        if controlNewElement(degreeElement, z) == False:
#             print "Old element ", z, " can be rewritten "
            deleteRewritableElement(stringNonRewritable.index(z))
            
def controlNontrivReprezentace(A):#checks if for beta real there exists a non-trivial 0 representation
    print "Control of non-trivial reprezentation of zero:"
    minA = min(A)
    maxA = max(A)
    if beta_complex(beta) <= max([maxA+1,-minA+1]):
        print "Has non-trivial reprezentation"
        return true
    else:
        print "Does not have non-trivial reprezentation"
        return false
    
def computeH(A): #program/function to compute an accurate value of parameter H in the algorithm for preprocessing
    global H
    degree = 0
    setM = []
    setM = list(A)
    numPreviousElements = []
    numPreviousElements.append(len(A))
    sumNumPreviousElements = 0
    string = []
    if cPisot == 1:
        for i in range(1, 10):
            print beta_complex(beta**i)
            if beta_complex(beta**i).imag() == 0:
                degree = i
                print "We calculate H to the degree ", degree
                break
        aux = [0 for i in range(degree +1)]
        for i in A: 
            aux[0] = i
            string.append(list(aux))
        for j in range(1, degree):
            for element in range(sumNumPreviousElements, numPreviousElements[j-1]):
                for a in A:
                    setM.append(setM[element]*beta+a)
            sumNumPreviousElements = sumNumPreviousElements + numPreviousElements[j-1]
            numPreviousElements.append(len(setM)-sumNumPreviousElements)
            print sumNumPreviousElements
            print numPreviousElements
        biggest = max(map(abs, map(beta_complex, setM)))
        H = biggest/(abs(beta_complex(beta**degree))-1) #accurate value of H
        print H
        for i in setM: #finding of the largest element which was used in computing of H
            if abs(beta_complex(i)) == biggest:
                print "biggest element in the set setM is ", i
    else:                 #if beta is not complex Pisot
        H = max(map(beta_complex, A))/(abs(beta_complex(beta))-1)
    
def getDforNetrivRepres(A):#computes the value of Dmin if there exists only trivial representation of 0
    minA = min(A)
    global D
    D = beta_complex((beta-1+minA)/(beta*(beta-1)))
    print beta_complex((beta-1+minA)/(beta*(beta-1)))
            
def getSpectrumForUptoM(spectrum, A, H, maxDegree): #generating of the spectrum upto the power BETA^m
    file = open("pravidla.txt", "w")
    file.write("Rules for polynomial " + str(f) + "\n")
    global D
    global epsilon
    global numPositions
    global allLL
    global allL
    global LL
    global L
    global LonlyPositive
    global LLnewLowerThanD
    global LLlowerThanD
    
    sumElementsLowerDegree = 0
    print "sumElementsLowerDegree ", sumElementsLowerDegree
    numNewElements.append(len(spectrum))
    
    smallestString = H
    print "The smallest string is ", beta_complex(smallestString)
    
    for degree in range(1,maxDegree+1):
        print "Number of elements in the spectrum is ", len(spectrum)
        print "Number of new elements ", numNewElements
        allLL = allLL + [i for i in LL if i not in allLL]
        allL = allL + [i for i in L if i not in allL]
        print "Number of positive rules :", len(allLL)
        epsilon = 1000
        print ""
        m = degree +1
        print ""
        print "##########################################################################################"

        print ""
        for k in range(sumElementsLowerDegree, sum(numNewElements)): #iteration over the existing elements from previous iteration
            for i in A: 
                new = 1
                for j in spectrum: #comparison of ki with every element of the spectrum
                    if beta*spectrum[k] + i == j: #comparison of the value
                        new = 0
                        if SS[spectrum.index(j)][degree]==0: #if they are not of the same length, rewriting rule is saved
#                             if (beta_complex(SS[k][degree-1]) >=1) or (beta_complex(SS[k][degree-1] - beta-1) ==0 or beta_complex(SS[k][degree-1]-1/(beta+1))==0): 
#                             if (beta_complex(SS[k][degree-1]) >=1) or (beta_complex(SS[k][degree-1] - beta-1) ==0):
                            if (beta_complex(SS[k][degree-1]) >=1): #only rewriting rules starting on a positive number are saved (if the alphabet is complex, uncomment previous)
                                writeRule(j, k, i, degree)
                            elif SS[k][degree-1]<0:
                                L.append([beta*spectrum[k]+i, degree+1])
                        break
                if new == 1:
                    if abs(beta_complex(beta*spectrum[k]+i)) !=0:
#                             print abs(beta_complex(beta*spectrum[k]+i))- H
                        if (SS[k][degree-1]!=0) and (abs(beta_complex(beta*spectrum[k]+i)) - H < epsilon):
                            epsilon = abs(beta_complex(beta*spectrum[k]+i))- H
                            print "changes to ", epsilon
                            print SS[k], " + ", i
#                             else:
#                                 print SS[k], " + ", i, " is out of K, value ", abs(beta_complex(beta*spectrum[k]+i)), abs(beta_complex((beta*spectrum[k]+i)/(beta**(degree+1))))
                        addNewElement(k, i, degree)
        controlAllElements()                      
        sumElementsLowerDegree = sum(numNewElements) #actual number of elements from previous iterations
        numNewElements.append(len(spectrum)-sumElementsLowerDegree) #adding of the number of elements from the last iteration
        print ""
        print "epsilon = ", epsilon, " m = ", m
        if epsilon < 0: #if epsilon is <0, Dmin will be <0 -> one more iteration
            print "INVALID D !!!!!!!!!!"
        else:
            print "VALID D"

        D = min(min(map(abs, setNonRewritable)), epsilon/(abs(beta_complex((beta)^m)))) #checks if rewriting rules can be omitted or not (if they have value greater than Dmin)
        LLnewLowerThanD = [i for i in LL if ((abs(beta_complex(LonlyPositive[LL.index(i)][0]/(beta**LonlyPositive[LL.index(i)][1]))) - smallestString/abs(beta_complex(beta**L[LL.index(i)][1]))<=D) and (i not in LLlowerThanD))]
        print LLnewLowerThanD
        LLlowerThanD = LLlowerThanD + LLnewLowerThanD
        print LLlowerThanD
            
        print "Number of rewriting rules smaller then Dmin is: ", len(LLlowerThanD)
            
        print ""
        print "Number of rewriting rules is:", len(LL) #total number of rewriting rules found until now
        file.write("\n" + "m = %d \n \n" % m)
        temp = [i for i in LL if (i not in allLL)] #writes all new rewriting rules for the power m
        for i in temp:
            print(i)
            file.write(i + "\n") #writes rewriting rule into the file
        print ""
        
        print "m = ", m
        print "value D = ", D
        print ""
        smallestElement = min(map(abs, setNonRewritable))
        print " min(map(abs, setNonRewritable)) = ", smallestElement
        print " epsilon/(abs(beta)^m)) = ",epsilon, " / abs(beta)^", m , " = ", epsilon/(abs(beta_complex((beta)^m)))  

        for i in setNonRewritable:
            if abs(i) == smallestElement:
                print "The smallest element in the set of non-rewritable is ", stringNonRewritable[setNonRewritable.index(i)]
    file.close()    

print ""
if (pisot == 1) or (cPisot==1): 
    print "max of the alphabet is ", max(map(beta_complex,A))
    print "value H:"
    computeH(A)
    print "rough estimate of H would be ", max(map(beta_complex, A))/(abs(beta_complex(beta))-1) #comparison with a rough estimate of H
    print "H = ", H ," with accurate value ", H

    spectrum = [] #set of elements of the spectrum
    SS = [] #set of values of the elements of the spectrum

########################################

    numPositions = 12 #bound for number of digits in the strings

########################################

    setNonRewritable = [] #list of elements of spectrum which are inside the circle H + epsilon which can not be rewritten
    stringNonRewritable = []
    aux = [0 for i in range(numPositions)]
    for k in A:
        if k != 0: #we do not consider 0 
            aux[0] = k
            stringNonRewritable.append(list(aux))
            setNonRewritable.append(beta_complex(k / beta))

    aux = [0 for i in range(numPositions)]
    for i in A: #adds alphabet to the sets
#         if i >= 0:
        aux[0] = i
        spectrum.append(i)
        SS.append(list(aux))
    
    LL = [] #list of rewriting rules
    L = [] #list of values of every rewriting rule (even starting on a negative digit)
    LonlyPositive = [] #list of values of elements of LL
    LLnewLowerThanD = [] #list of new rewriting rules smaller than Dmin
    LLlowerThanD = [] #list of all rewriting rules smaller than Dmin
    allL = []
    allLL = []
    numNewElements = []

    if (controlNontrivReprezentace(A) == true) or (cPisot ==1):
        getSpectrumForUptoM(spectrum, A, H, 5)
    else: 
        D = 0
        getDforNetrivRepres(A)
        
        print "value D = ", D
            
print "FINISHED"

 v = omega, w = omega^2
Alphabet A =  [0, 1, 2, -1, -2]

Is irreducible:  True
Number Field in beta with defining polynomial x^2 - 2*x - 1
[-0.414213562373095, 2.41421356237309]
Beta is Pisot
[Ring morphism:
  From: Number Field in beta with defining polynomial x^2 - 2*x - 1
  To:   Real Field with 106 bits of precision
  Defn: beta |--> -0.4142135623730950488016887242097, Ring morphism:
  From: Number Field in beta with defining polynomial x^2 - 2*x - 1
  To:   Real Field with 106 bits of precision
  Defn: beta |--> 2.414213562373095048801688724210]
beta =  2.414213562373095048801688724210
Alphabet A =  [0, 1, 2, -1, -2]

max of the alphabet is  2.000000000000000000000000000000
value H:
rough estimate of H would be  1.414213562373095048801688724210
H =  1.414213562373095048801688724210  with accurate value  1.414213562373095048801688724210
Control of non-trivial reprezentation of zero:
Has non-trivial reprezentation
sumElementsLowerDegree  0
The smallest string is  1.41421356237309504

changes to  65.94112549695428117124052938103
[2, -2, 2, -1, 1, 0, 0, 0, 0, 0, 0]  +  -2
changes to  31.97056274847714058562026469052
[2, -2, 2, -2, 1, 0, 0, 0, 0, 0, 0]  +  -2

epsilon =  31.97056274847714058562026469052  m =  6
VALID D
['1-22-220 -> 00211-1', '1-22-22-1 -> 00211-2']
['1-1-1 -> 010', '1-1-2 -> 01-1', '1-20 -> 001', '1-21 -> 002', '1-2-1 -> 000', '1-2-2 -> 00-1', '1-220 -> 011-1', '1-10 -> 011', '1-221 -> 0110', '1-22-1 -> 011-2', '1-22-20 -> 00122', '1-22-21 -> 00202', '1-22-2-1 -> 00121', '1-22-2-2 -> 00120', '1-22-220 -> 00211-1', '1-22-22-1 -> 00211-2']
Number of rewriting rules smaller then Dmin is:  16

Number of rewriting rules is: 90
1021-10 -> 022221
1021-11 -> 022222
1021-1-1 -> 022220
1021-1-2 -> 02222-1
1021-20 -> 022211
1021-21 -> 022212
1021-2-1 -> 022210
1021-2-2 -> 02221-1
102-120 -> 02221-1
102-121 -> 022210
102-122 -> 022211
102-12-1 -> 02221-2
102-220 -> 02211-1
102-221 -> 022110
102-222 -> 022111
102-22-1 -> 02211-2
11-12-20 -> 022122
11-12-21 -> 022